In [ ]:
!pip install livelossplot

In [ ]:
import numpy as np
import pandas as pd
import random as rand
from IPython.display import Image, display

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from livelossplot import PlotLossesKerasTF

from google.colab import drive
from google.colab import files

from sklearn.model_selection import train_test_split

import seaborn as sns
import cv2

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,MaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

In [ ]:
# upload kaggle.json
from google.colab import files
files.upload()

In [ ]:
# make directory and change permission
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

In [ ]:
# download dataset, choose 'copy api command' from kaggle dataset
!kaggle datasets download -d alxmamaev/flowers-recognition

In [ ]:
# unzip
import zipfile
import os
import shutil

file_zip = 'flowers-recognition.zip'
zip_ref = zipfile.ZipFile(file_zip, 'r')
zip_ref.extractall()
zip_ref.close()

listdir_image = os.listdir('./flowers/')
print(listdir_image)

In [ ]:
# delete dir that is not used
import shutil

dir_image ='./flowers/'
list_category = ['dandelion', 'tulip', 'rose', 'sunflower', 'daisy']

for x in listdir_image:
  if x not in list_category:
    path = os.path.join(dir_image, x)
    if os.path.isfile(path):
        os.remove(path)
    elif os.path.isdir(path):
        shutil.rmtree(path)
    else:
        print("Unknown or already remove!")

listdir_image = os.listdir(dir_image)
listdir_image

In [ ]:
# Menampilkan jenis bunga
img = plt.imread("./flowers/daisy/5547758_eea9edfd54_n.jpg")
img = cv2.resize(img,(255,255))
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
#Menentukan path dataset yang ada di GDrive
BASE_DIR = './flowers/'

In [ ]:
flowers_dict = {}
for flower in os.listdir(BASE_DIR):
    folder_path = os.path.join(BASE_DIR, flower)
    flowers = os.listdir(folder_path)
    flowers_dict[flower] = [folder_path, flowers]
    img_idx = rand.randint(0,len(flowers)-1)
    flwr_img_path = os.path.join(BASE_DIR, flower, flowers[img_idx])
    print(flwr_img_path)
    print('Name of Flower: ', flower, u'\u2193', 'at index: ', img_idx)
    display(Image(filename=flwr_img_path))

In [ ]:
#Variabel global
IMG_SIZE = 255
BATCH_SIZE = 64

In [ ]:
# Generate data
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest',
    validation_split=0.2
    )

In [ ]:
# Generate data latih
train_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size= (IMG_SIZE, IMG_SIZE),
    batch_size= BATCH_SIZE,
    class_mode='categorical',
    subset='training',  # Mengatur sebagai train data
    shuffle= True
    ) 

In [ ]:
# Generate data validasi
val_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size= (IMG_SIZE, IMG_SIZE),
    batch_size= BATCH_SIZE,
    class_mode= 'categorical',
    subset= 'validation',  # Mengatur sebagai val data
    shuffle= False
    )

In [ ]:
 # Membuat model CNN
 model = tf.keras.models.Sequential([
    # Layer 1
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2, 2, padding='same'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    # Layer 2
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2, padding='same'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    # Layer 3
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2, padding='same'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    # Layer 4
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2, padding='same'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    # # Layer 5
    # tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    # tf.keras.layers.MaxPooling2D(2, 2, padding='same'),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Flatten(),
    # Fully Connected Layer 1
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    # tf.keras.layers.BatchNormalization(),
    #Add output layer
    tf.keras.layers.Dense(5, activation="softmax")
])

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
# Membuat callback
path_acc='/content/gdrive/My Drive/Kaggle/best_model.hdf5'

# Membuat model checkpoint untuk menyimpan best acc
checkpoint_acc = ModelCheckpoint(path_acc, verbose=1, monitor='val_accuracy', save_best_only=True, mode='max')

# Early stop train apabila val loss bertambah
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

# Mengurangi lr apabila val acc tidak bertambah
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', mode='max', factor=0.1, patience=3, min_delta=1e-4, verbose=1)
callbacks_list = [checkpoint_acc,
                  early_stopping, 
                  lr_reduce]     

In [ ]:
# Proses melatih data
STEPS_PER_EPOCH = train_generator.n//train_generator.batch_size
VALIDATION_STEPS = val_generator.n//val_generator.batch_size

history = model.fit(
    train_generator, 
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=200, 
    validation_data=val_generator,
    validation_steps=VALIDATION_STEPS, 
    verbose=1,
    callbacks=callbacks_list
)

In [ ]:
# Menampilkan hasil evaluasi
eval_val = model.evaluate(val_dataset, steps=8) 
print("\nVal Loss " + str(eval_val[0]))
print("Val Acc: " + str(eval_val[1]))

In [ ]:
# Menampilkan diagram hasil evaluasi loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# Menampilkan diagram hasil evaluasi akurasi
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# Menguji model
model = load_model(path_acc) 
uploaded = files.upload()

for fn in uploaded.keys():
  path = fn
  img = image.load_img(path, target_size=(255,255))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=32)
  predIdxs = np.argmax(classes, axis=1)
  print(classes)
  print(fn)
  print(predIdxs)
  # if classes[0, 0] != 0:
  #   print('Predict : Paper')
  # elif classes[0, 1] != 0:
  #   print('Predict : Rock')
  # else:
  #   print('Predict : Scissors')


In [ ]:
# Konversi model ke TFlite/Android.
model = load_model(path_acc) 
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Menyimpan model ke TFlite/Android.
with open('/content/gdrive/My Drive/Kaggle/model-for-android.tflite', 'wb') as f:
  f.write(tflite_model)